In [457]:
from itertools import combinations
from z3 import *
from utils import *
import math
from timeit import default_timer as timer

### naive pairwise


In [458]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one_np(bool_vars):
    return And(at_least_one_np(bool_vars), And(at_most_one_np(bool_vars)))



def at_least_k_np(bool_vars, k):
    return at_most_k_np([Not(var) for var in bool_vars], len(bool_vars)-k)

def at_most_k_np(bool_vars, k):
    return And([Or([Not(x) for x in X]) for X in combinations(bool_vars, k + 1)])

def exactly_k_np(bool_vars, k):
    return And(at_most_k_np(bool_vars, k), at_least_k_np(bool_vars, k))

### sequential

In [459]:
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars))



def at_least_k_seq(bool_vars, k):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k)

def at_most_k_seq(bool_vars, k):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))   
    return And(constraints)

def exactly_k_seq(bool_vars, k):
    return And(at_most_k_seq(bool_vars, k), at_least_k_seq(bool_vars, k))

### bitwise

In [460]:
def toBinary(num, length = None):
    num_bin = bin(num).split("b")[-1]
    if length:
        return "0"*(length - len(num_bin)) + num_bin
    return num_bin
    
def at_least_one_bw(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_bw(bool_vars):
    constraints = []
    n = len(bool_vars)
    m = math.ceil(math.log2(n))
    r = [Bool(f"r_{i}") for i in range(m)]
    binaries = [toBinary(i, m) for i in range(n)]
    for i in range(n):
        for j in range(m):
            phi = Not(r[j])
            if binaries[i][j] == "1":
                phi = r[j]
            constraints.append(Or(Not(bool_vars[i]), phi))        
    return And(constraints)

def exactly_one_bw(bool_vars):
    return And(at_least_one_bw(bool_vars), at_most_one_bw(bool_vars)) 

### heule

In [461]:
def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars):
    if len(bool_vars) <= 4:
        return And(at_most_one_np(bool_vars))
    y = Bool(f"y")
    return And(And(at_most_one_np(bool_vars[:3] + [y])), And(at_most_one_he(bool_vars[3:] + [Not(y)])))

def exactly_one_he(bool_vars):
    return And(at_most_one_he(bool_vars), at_least_one_he(bool_vars))

# Model

#### get the data

In [462]:
def read_file(in_file):
    f = open(in_file, "r")
    lines = f.read().splitlines()
    
    w = int(lines[0])
    n = int(lines[1])
    
    x = []
    y = []
    for i in range(int(n)):
            split = lines[i + 2].split(' ')
            x.append(int(split[0]))
            y.append(int(split[1]))
            
    maxlen = sum(y)
    return w, n, x, y, maxlen

w, n, x, y, maxlen = read_file(".\instances\ins-10.txt")

print(w,n,x,y,maxlen)

biggest_silicon = 0
for k in range(n):
    if x[biggest_silicon]*y[biggest_silicon] < x[k]*y[k]:
        biggest_silicon = k
print(biggest_silicon)

17 12 [3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 7, 7] [3, 4, 5, 6, 7, 8, 9, 3, 8, 14, 3, 6] 76
9


#### define the variables

In [463]:
solution = [[[Bool(f"solution_{i}_{j}_{k}") for k in range(n)] for j in range(maxlen)] for i in range(w)]   
#k is the number of the silicon
#i is the possible positions in w
#j is the possible positions in l

#### choice of the method

In [464]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one(bool_vars):
    return at_most_one(bool_vars) + [at_least_one(bool_vars)]

#### constraints

In [465]:
def solve_problem(w,n,x,y,maxlen,l):
    start = timer()
    
    solved = False
    while l<=maxlen and solved==False:
        s = Solver()
        #no overlapping
        for j in range(l):
            for i in range(w):
                s.add(at_most_one([solution[i][j][k] for k in range(n)]))

        #makes sure silicons fit            
        for k in range(n):
            possible_sols = []
            for i in range(w-x[k]+1):
                for j in range(l-y[k]+1):
                    circuit = []
                    for ox in range(w):
                        for oy in range(l):
                            if i<=ox<i+x[k] and j<=oy<j+y[k]:
                                circuit.append(solution[ox][oy][k])
                            else:
                                circuit.append(Not(solution[ox][oy][k]))
                    possible_sols.append(And(circuit))
            s.add(exactly_one(possible_sols))
            
        #puts the silicon with larger area in the bottom left corner    
        s.add([And(solution[0][0][biggest_silicon])])

        if s.check() == sat:
            time = timer() - start
            print("model solved with length:", l, "in time: ", time, "s")
            print(s.model())
            solved = True
        else:
            print("Failed to solve with length: ", l)
            l = l+1   
    return s.model(), l

In [466]:
%%time
l = min(y)
model, l = solve_problem(w,n,x,y,maxlen,l)

Failed to solve with length:  3
Failed to solve with length:  4
Failed to solve with length:  5
Failed to solve with length:  6
Failed to solve with length:  7
Failed to solve with length:  8
Failed to solve with length:  9
Failed to solve with length:  10
Failed to solve with length:  11
Failed to solve with length:  12
Failed to solve with length:  13
Failed to solve with length:  14
Failed to solve with length:  15
Failed to solve with length:  16
model solved with length: 17 in time:  241.29964450000261 s
[solution_13_14_1 = False,
 solution_14_13_3 = False,
 solution_1_9_7 = False,
 solution_16_2_3 = False,
 solution_3_2_0 = False,
 solution_0_5_6 = False,
 solution_5_14_6 = False,
 solution_16_14_7 = False,
 solution_5_2_10 = False,
 solution_13_15_11 = False,
 solution_13_2_0 = False,
 solution_14_0_5 = False,
 solution_7_1_2 = False,
 solution_0_0_2 = False,
 solution_15_2_3 = False,
 solution_9_12_1 = False,
 solution_12_16_8 = False,
 solution_1_12_11 = False,
 solution_0_15_

In [467]:
def get_solution(model, solution, w, l, n, r=None):
    # Create solution array
    solution = np.array([[[is_true(model[solution[i][j][k]]) for k in range(n)] for j in range(maxlen)] for i in range(w)])
    p_x_sol = []
    p_y_sol = []
    rot_sol = [False for i in range(n)]

    for k in range(n):
        x_ids, y_ids = solution[:, :, k].nonzero()
        x = np.min(x_ids)
        y = np.min(y_ids)
        p_x_sol.append(x)
        p_y_sol.append(y)
        if r is not None:
            rot_sol[k] = is_true(model[r[k]])
    return p_x_sol, p_y_sol, rot_sol, l

In [468]:
get_solution(model, solution, w, l, n, r=None)

([7, 14, 4, 7, 14, 11, 4, 10, 7, 0, 0, 10],
 [14, 6, 0, 0, 10, 6, 5, 14, 6, 0, 14, 0],
 [False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False],
 17)